In [1]:
import os


In [2]:
!pwd

/Users/shrey/Desktop/github/butterfly_classifier/research


In [3]:
os.chdir('../')
!pwd

/Users/shrey/Desktop/github/butterfly_classifier


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dir

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_dir([self.config.artifacts_root])
                 


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import boto3
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size



In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.s3 = boto3.client('s3')
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3_url = s3_url = self.config.source_url
            logger.info(f'Source URL: {type(s3_url)}, value: {s3_url}')

            # Ensure the source_url is a string
            if not isinstance(s3_url, str):
                raise ValueError(f"source_url must be a string. Got {type(s3_url)} instead.")


            bucket_name = s3_url.split('/')[2]
            object_key = '/'.join(s3_url.split('/')[3:])

            logger.info(f"Downloading file from s3 bucket: {bucket_name}, object_key: {object_key}")

            with open(self.config.local_data_file, 'wb') as f:
                self.s3.download_fileobj(bucket_name, object_key, f)
            logger.info(f"File downloaded at: {self.config.local_data_file}")
            # filename, headers = request.urlretrieve(
            #     url=self.config.source_url,
            #     filename=self.config.local_data_file
            # )
            # logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file to the data dir (unzip_dir)
        Function returns None
        """ 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Error in data ingestion: {e}")
    raise e

[2025-01-27 14:31:03,008: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-27 14:31:03,009: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-27 14:31:03,010: INFO: common: Directory created successfully: artifacts]
[2025-01-27 14:31:03,010: INFO: common: Directory created successfully: artifacts/data_ingestion]
[2025-01-27 14:31:03,053: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]
[2025-01-27 14:31:03,143: INFO: 1154599334: Source URL: <class 'str'>, value: s3://myflydatasetbucket/data.zip]
[2025-01-27 14:31:03,143: INFO: 1154599334: Downloading file from s3 bucket: myflydatasetbucket, object_key: data.zip]
[2025-01-27 14:31:03,710: INFO: httpchecksum: Skipping checksum validation. Response did not contain one of the following algorithms: ['crc64nvme', 'crc32c', 'crc32', 'sha1', 'sha256'].]
[2025-01-27 14:31:03,842: INFO: httpchecksum: Skipping checksum validation. Response did not contain one of the fol